## IntCode

In [2]:
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    prog.state = 0
    run(prog)
end

run(prog::Prog, input::Int) = run(prog, [input])

function run(prog)
    clear!(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear!(prog) = (prog.output = [])

clear! (generic function with 1 method)

## Part 1

In [6]:
vm = Prog(readline("input.txt"))
img_txt = run(vm, [50, 50])

1-element Array{Int64,1}:
 0

In [170]:
arr = zeros(Int, 200, 200)
for idx in CartesianIndices(arr)
    vm = Prog(readline("input.txt"))
    arr[idx] = run(vm, [idx[1] - 1, idx[2] - 1])[1]
end
sum(arr)

2871

## Part 2

In [160]:
for i in 50:-1:1
    println(join(map(x -> x == 0 ? '.' : '#', arr[i, :])))
end

..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
.................................................#
................................................##
...............................................###
.............................................#####
............................................######
...............................

In [48]:
minimum(findall(arr[:, 50] .== 1))

30

In [178]:
function get_line(arr, func)
    line = Dict{Int, Int}()
    for i in 1:size(arr)[1]
        if !isempty(findall(arr[:, i] .== 1))
            line[i - 1] = func(findall(arr[:, i] .== 1)) - 1
        end
    end
    
    line
end

lines = [get_line(arr, func) for func in [minimum, maximum]]

function showline(l, func)
    xx = sort(collect(keys(l)))
    for x in xx
        println(x, ":", l[x], ":", func(x))
    end
end

function err(l, a)
    xx = sort(collect(keys(l)))
    sum([l[x] - f1(x, a) for x in xx])
end

err (generic function with 1 method)

In [1]:
f1(x, a) = Int(ceil(a*x))
f2(x, a) = Int(floor(a*x))

f2 (generic function with 1 method)

In [67]:
err(line, 0)

722

In [215]:
findall(arr[30, :] .== 1)

10-element Array{Int64,1}:
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51

In [79]:
err(line, 1)

function dihotomy(line, a, b)
    v1 = err(line, a)
    v2 = err(line, b)
    v0 = v1
    c = a
    while  v0 != 0
        c = (a + b)/2
        v0 = err(line, c)
        if v0 > 0 
            a = c
        else
            b = c
        end
    end
    
    c
end

dihotomy (generic function with 1 method)

In [129]:
dihotomy(line, 0, 1)

0.572265625

In [126]:
showline(line, 0.573)

1:1:1
4:3:3
6:4:4
7:5:5
8:6:6
9:6:6
10:7:7
11:7:7
12:8:8
13:8:8
14:9:9
15:10:10
16:10:10
17:11:11
18:11:11
19:12:12
20:12:12
21:13:13
22:14:14
23:14:14
24:15:15
25:15:15
26:16:16
27:16:16
28:17:17
29:18:18
30:18:18
31:19:19
32:19:19
33:20:20
34:20:20
35:21:21
36:22:22
37:22:22
38:23:23
39:23:23
40:24:24
41:24:24
42:25:25
43:26:26
44:26:26
45:27:27
46:27:27
47:28:28
48:28:28
49:29:29
50:30:30
51:30:30
52:31:31
53:31:31
54:32:32
55:32:32
56:33:33
57:34:34
58:34:34
59:35:35
60:35:35
61:36:36
62:36:36
63:37:37
64:38:38
65:38:38
66:39:39
67:39:39
68:40:40
69:40:40
70:41:41
71:42:42
72:42:42
73:43:43
74:43:43
75:44:44
76:44:44
77:45:45
78:46:46
79:46:46
80:47:47
81:47:47
82:48:48
83:48:48
84:49:49
85:50:50
86:50:50
87:51:51
88:51:51
89:52:52
90:52:52
91:53:53
92:54:54
93:54:54
94:55:55
95:55:55
96:56:56
97:56:57
98:57:57
99:58:58
100:58:58


In [111]:
function dihotomy1(v0, a1, a2, f, eps)
    v1 = f(a1)
    v2 = f(a2)
    c = a1
    while abs(a1 - a2) > eps
        c = (a1 + a2)/2
        if f(c) > v0 
            a2 = c
        else
            a1 = c
        end
    end
    return c
end

function dihotomy2(v0, a1, a2, f, eps)
    v1 = f(a1)
    v2 = f(a2)
    c = a1
    while abs(a1 - a2) > eps
        c = (a1 + a2)/2
        if f(c) >= v0 
            a2 = c
        else
            a1 = c
        end
    end
    return c
end

dihotomy1(line[47], 0, 1, x -> f1(47, x), 10^-10)

0.5978260869742371

In [172]:
xx = sort(collect(keys(line)))
println(minimum([dihotomy1(line[x], 0, 1, z -> f1(x, z), 10^-10) for x in xx]))
println(maximum([dihotomy2(line[x], 0, 1, z -> f1(x, z), 10^-10) for x in xx]))

#dihotomy2(line[50], 0, 1, x -> f1(50, x), 10^-10)

0.5744680851348676
0.571428571420256


In [ ]:
# 0.573



In [155]:
showline(lines[2], x -> f2(x, 0.7158385093207471))

0:0:0
3:2:2
5:3:3
6:4:4
7:5:5
8:5:5
9:6:6
10:7:7
11:7:7
12:8:8
13:9:9
14:10:10
15:10:10
16:11:11
17:12:12
18:12:12
19:13:13
20:14:14
21:15:15
22:15:15
23:16:16
24:17:17
25:17:17
26:18:18
27:19:19
28:20:20
29:20:20
30:21:21
31:22:22
32:22:22
33:23:23
34:24:24
35:25:25
36:25:25
37:26:26
38:27:27
39:27:27
40:28:28
41:29:29
42:30:30
43:30:30
44:31:31
45:32:32
46:32:32
47:33:33
48:34:34
49:35:35


In [181]:
res = []
for line in lines
    xx = sort(collect(keys(line)))
    a1 = maximum([dihotomy2(line[x], 0, 1, z -> f1(x, z), 10^-10) for x in xx])
    a2 = minimum([dihotomy1(line[x], 0, 1, z -> f1(x, z), 10^-10) for x in xx])
    push!(res, (a1, a2, (a1 + a2)/2))
end

res

2-element Array{Any,1}:
 (0.571428571420256, 0.5721649485058151, 0.5717967599630356) 
 (0.7106598985265009, 0.6000000000349246, 0.6553299492807128)

In [179]:
lines[1]

Dict{Int64,Int64} with 197 entries:
  11  => 7
  134 => 77
  158 => 91
  160 => 92
  29  => 17
  131 => 75
  173 => 99
  74  => 43
  176 => 101
  57  => 33
  31  => 18
  70  => 41
  33  => 19
  114 => 66
  165 => 95
  96  => 55
  133 => 77
  49  => 29
  84  => 49
  117 => 67
  93  => 54
  50  => 29
  77  => 45
  80  => 46
  188 => 108
  ⋮   => ⋮

In [103]:
# 0.5729483282775618; f1
# 0.7158385093207471; f2

7.450580596923828e-9

In [2]:
function part2()
    i = 50
    while true
        if f2(i - 99, 0.7163649835856631) < f1(i, 0.5717967599630356) + 99
            i += 1
        else
            return i
        end
    end
end

part2 (generic function with 1 method)

In [3]:
part2()

1180

In [255]:
f1(1180, 0.5717967599630356), f2(1180 - 99, 0.7163649835856631)

(675, 774)

In [256]:
1180 - 99

1081

In [158]:
f1(50, 0.5729483282775618)

29

In [161]:
f2(40, 0.7158385093207471)

28

In [187]:
1089*10000 + 680

10890680

In [188]:
1089

1089

In [207]:
res = []
for i in 0:29
    vm = Prog(readline("input.txt"))
    push!(res, run(vm, [30, i])[1])
end

In [208]:
findall(res .== 1)

0-element Array{Int64,1}

In [251]:
vm = Prog(readline("input.txt"))
run(vm, [779, 1087])

1-element Array{Int64,1}:
 0

In [231]:
680*10000 + 1089

6801089

In [ ]:
6791088

In [ ]:
6751081